# Particle Tracking with PRT

## Setup

### Packages

In [ ]:
from pathlib import Path

import flopy as fp
import numpy as np
from shapely.geometry import Polygon
import geopandas as gpd
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import nlmod

### Logging

In [ ]:
nlmod.util.get_color_logger("INFO")
nlmod.show_versions()

### Load base model

In [ ]:
model_ws = Path("./scratch_model")
model_name = "from_scratch"

ds = xr.open_dataset(model_ws / f"{model_name}.nc")

# constants
wells = pd.DataFrame(
    [[100, -50, -5, -10, -100.0], [200, 150, -20, -30, -300.0]],
    columns=["x", "y", "top", "botm", "Q"],
    index=pd.Index([0, 1], name="well no."),
)
xyriv = [
    (250.0, -500.0),
    (300.0, -300.0),
    (275.0, 0.0),
    (200.0, 250.0),
    (175.0, 500.0),
]

## PRT

### Forward

In [ ]:
# create new subdir for PRT model
prt_ws = Path(model_ws) / "prt_fw"

# Create a new Simulation object
simprt = nlmod.sim.sim(ds, sim_ws=prt_ws)
_ = nlmod.sim.tdis(ds, simprt)

# Add PRT model
prt = nlmod.prt.prt(ds, simprt)

# DIS: discretization package
_ = nlmod.prt.dis(ds, prt)

# MIP: Model Input Package
_ = nlmod.prt.mip(ds, prt, porosity=0.3)

# PRP: particle release point package
# define particle release point  every 11th cell in the first layer
pdata = fp.modpath.ParticleData(
    [
        (k, i, j)
        for i in np.arange(0, ds.sizes["y"], 11)
        for j in np.arange(0, ds.sizes["x"], 11)
        for k in [0]
    ],
    structured=True,
)
release_pts = list(pdata.to_prp(prt.modelgrid, global_xy=False))
prp = nlmod.prt.prp(ds, prt, packagedata=release_pts, perioddata={0: ["FIRST"]})

# FMI: flow model interface
fmi = nlmod.prt.fmi(ds, prt)

# OC: output control
trackcsvfile_prt = f"{prt.name}.trk.csv"
oc = nlmod.prt.oc(
    ds,
    prt,
    trackcsv_filerecord=trackcsvfile_prt,
)

# EMS: explicit model solution
ems = nlmod.sim.ems(simprt, model=prt)

In [ ]:
simprt.write_simulation(silent=True)
simprt.run_simulation(silent=True)

#### Load pathline data

The read_pathlines function is a wrapper around pandas.read_csv but it adds the particle id for you. Additionallly it is usefull as documentation on the different istatus and ireason types.

In [ ]:
nlmod.prt.read_pathlines?

In [ ]:
pathlines_fw = nlmod.prt.read_pathlines(prt_ws / trackcsvfile_prt)
pathlines_fw.head()

#### Plot results

In [ ]:
f, ax = plt.subplots(figsize=(8, 8))
pmv = fp.plot.PlotMapView(prt, ax=ax)
pmv.plot_grid(lw=0.1, color="k")
pmv.plot_pathline(
    pathlines_fw,
    layer="all",
    colors="0.7",
    lw=1.0,
)
for ilay in [0, 1, 2]:
    mask = pathlines_fw["ilay"] == ilay
    pmv.plot_pathline(
        pathlines_fw.loc[mask],
        layer=ilay - 3,
        lw=0.0,
        marker=".",
        ms=4,
        markercolor=f"C{ilay}",
        markerevery=5,
    )
pmv.plot_endpoint(pathlines_fw, color="k", marker=".", direction="starting", zorder=10)


# plot river and wells
def plot_river_and_wells(ax: plt.Axes):
    ax.plot(
        [xy[0] for xy in xyriv],
        [xy[1] for xy in xyriv],
        "b-",
        lw=2,
        label="river",
    )
    ax.plot(wells["x"].values, wells["y"].values, "rs", ms=5)


plot_river_and_wells(ax=pmv.ax)
handles = [
    pmv.ax.plot([], [], "C0.", ms=5, label="Layer 0")[0],
    pmv.ax.plot([], [], "C1.", ms=5, label="Layer 1")[0],
    pmv.ax.plot([], [], "C2.", ms=5, label="Layer 2")[0],
]
labels = [h.get_label() for h in handles]
pmv.ax.legend(handles, labels, loc=(0, 1), frameon=False, ncol=3)
pmv.ax.set_xlabel("X [m]")
pmv.ax.set_ylabel("Y [m]");

### Backward and refined

#### Refine and run gwf model

In [ ]:
refinement_extent = [-100.0, 200.0, -200.0, 300.0]
refinement_features = (
    [
        nlmod.util.extent_to_polygon(refinement_extent),
    ],
    "polygon",  # type of feature
    1,  # refinement level
)
dsr = nlmod.grid.refine(ds, refinement_features=[refinement_features])

In [ ]:
dsr.attrs["model_name"] = f"{ds.attrs['model_name']}_ref"
dsr.attrs["model_ws"] = f"./{ds.attrs['model_ws']}_ref"

In [ ]:
sim = nlmod.sim.sim(dsr)
tdis = nlmod.sim.tdis(dsr, sim)
ims = nlmod.sim.ims(sim, complexity="SIMPLE")
gwf = nlmod.gwf.gwf(dsr, sim)
disv = nlmod.gwf.disv(dsr, gwf)
npf = nlmod.gwf.npf(
    dsr, gwf, save_flows=True, save_specific_discharge=True, save_saturation=True
)
ic = nlmod.gwf.ic(dsr, gwf, starting_head=1.0)
oc = nlmod.gwf.oc(dsr, gwf, save_head=True)
wel = nlmod.gwf.wells.wel_from_df(wells, gwf)

riv_layer = 0  # add to first layer
bed_resistance = 0.1  # days
riv_stage = 1.0  # m NAP
riv_botm = -3.0  # m NAP
riv_data = nlmod.gwf.surface_water.rivdata_from_xylist(
    gwf, xyriv, riv_layer, riv_stage, bed_resistance, riv_botm
)


def get_riv_cond_from_cid(icell2d: int, ds: xr.Dataset, bed_resistance: float) -> float:
    """Get river conductance from cell id."""
    area = ds.area.sel(icell2d=icell2d).values
    # calculate conductance
    return area / bed_resistance


riv_datac = [
    [x[0], x[1], get_riv_cond_from_cid(x[0][1], dsr, x[2]), x[3]] for x in riv_data
]
riv = fp.mf6.ModflowGwfriv(gwf, stress_period_data={0: riv_datac})

nlmod.sim.write_and_run(sim, dsr, silent=False)

#### Reverse heads and cellbudget files

In [ ]:
hds = nlmod.gwf.get_headfile(ds=dsr, gwf=gwf, tdis=tdis)
hds_bw_path = hds.filename.parent / f"{hds.filename.stem}_bkwd{hds.filename.suffix}"
hds.reverse(hds_bw_path)

cbc = nlmod.gwf.get_cellbudgetfile(ds=dsr, gwf=gwf, tdis=tdis)
cbc_bw_path = cbc.filename.parent / f"{cbc.filename.stem}_bkwd{cbc.filename.suffix}"
cbc.reverse(cbc_bw_path)

#### Create particle starts list

Start a particle in each (active) cell in refined extent

In [ ]:
dsr["z"] = (dsr["top"] + dsr["botm"]) / 2
dsr["idomain"] = (
    ("layer", "icell2d"),
    disv.idomain.array,
)  # make sure to get idomain otherwhise you might start particles in inactive cells and everything crashes #beentheredonethat

# map xy to cellids
nodes = dsr[["z", "idomain"]].where(
    (dsr.x >= refinement_extent[0])
    & (dsr.x <= refinement_extent[1])
    & (dsr.y >= refinement_extent[2])
    & (dsr.y <= refinement_extent[3]),
    drop=True,
)
ix = fp.utils.GridIntersect(gwf.modelgrid)
spatial_join = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(nodes.x, nodes.y),
).sjoin(
    gpd.GeoDataFrame(
        {"cellid": ix.cellids},
        geometry=ix.geoms,
    ),
    how="left",
)

# create release points
# define particle release point for every cell within refined extent
pid = 0
release_pts = []
for ilay in range(nodes.sizes["layer"]):
    nodes_lay = nodes.isel(layer=ilay)
    active = (nodes_lay["idomain"] > 0).values
    j = spatial_join["cellid"].values.astype(int)[active]
    k = np.full_like(j, ilay, dtype=int)
    x = nodes_lay["x"].values[active]
    y = nodes_lay["y"].values[active]
    z = nodes_lay["z"].values[active]
    pids = np.arange(pid, pid + len(k), dtype=int)
    pid += len(k)
    cellid = tuple(zip(k, j))
    release_ptsi = list(zip(pids, cellid, x, y, z))
    release_pts += release_ptsi

print(f"Number of release points: {len(release_pts)}")

In [ ]:
# create new subdir for PRT backwards model
prt_ws_bw = Path(dsr.attrs["model_ws"]) / "prt_bw"

# Create a new Simulation object
simprt_bw = nlmod.sim.sim(dsr, sim_ws=prt_ws_bw)
_ = nlmod.sim.tdis(dsr, simprt_bw)

# Add PRT model
prt_bw = nlmod.prt.prt(dsr, simprt_bw, modelname=dsr.model_name)

# Add DISV model
_ = nlmod.prt.disv(dsr, prt_bw)

# MIP: Model Input Package
_ = nlmod.prt.mip(dsr, prt_bw, porosity=0.3)

# PRP: particle release package
_ = nlmod.prt.prp(
    dsr,
    prt_bw,
    packagedata=release_pts,
    perioddata={0: ["FIRST"]},
)

# OC: output control
trackcsvfile_prt_bw = f"{prt_bw.name}bw.trk.csv"
_ = nlmod.prt.oc(dsr, prt_bw, trackcsv_filerecord=trackcsvfile_prt_bw)

# FMI: flow model interface
fmi_bw_packagedata = [
    ("GWFHEAD", hds_bw_path),
    ("GWFBUDGET", cbc_bw_path),
]
_ = nlmod.prt.fmi(dsr, prt_bw, packagedata=fmi_bw_packagedata)

# EMS: explicit model solution
_ = nlmod.sim.ems(simprt_bw, model=prt_bw)

# Write and run the simulation
simprt_bw.write_simulation()
success_bw_prt, _ = simprt_bw.run_simulation()

#### Load pathline data

In [ ]:
pathlines_bw = nlmod.prt.read_pathlines(
    prt_ws_bw / trackcsvfile_prt_bw, icell2d=dsr.icell2d.size
)

Get pathlines of particles that started in a well (backward)

In [ ]:
pathlines_bw0 = pathlines_bw.query("ireason==0")
well_pids = []
for _, row in wel.stress_period_data.dataframe[0].iterrows():
    cellid_layer = row["cellid_layer"]
    icell2d = row["cellid_cell"]
    well_pids.append(
        pathlines_bw0.query(f"ilay=={cellid_layer} and icell2d=={icell2d}")
        .loc[:, "pid"]
        .to_numpy()[0]
    )
pathlines_bww = pathlines_bw[pathlines_bw["pid"].isin(well_pids)]

#### Plot results

In [ ]:
f, ax = plt.subplots(figsize=(8, 8))
for pid, gr in pathlines_bww.groupby("pid"):
    for ilay, gr_lay in gr.groupby("ilay"):
        ax.plot(gr_lay["x"], gr_lay["y"], lw=1.0, color=f"C{ilay}", zorder=1)

nlmod.plot.modelgrid(dsr, ax=ax, linewidth=0.1)
plot_river_and_wells(ax=ax)
ax.set_xlim(ds.extent[0], ds.extent[1])
ax.set_ylim(ds.extent[2], ds.extent[3])
ax.set_aspect("equal")
ax.set_xlabel("X [m]")
ax.set_ylabel("Y [m]")